In [1]:
import pandas as pd
import numpy as np
import time
import sys

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

from lightgbm import LGBMClassifier
from sklearn.linear_model import LogisticRegression  
from sklearn import svm

os_env = 'mac'

if os_env == 'window':
    module_path='/Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Cohort_selection/'
else:
    module_path = '/Users/gwonjeong-eul/Desktop/ecp-scl-macbook/ECP_SCL/Cohort_selection/'

if module_path not in sys.path:
    sys.path.append(module_path)

from cohort_loader_new_version0229 import *
import eicu_year_process

if os_env == 'window':
    module_path='/Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Experiment(Supervised Learning)/evaluation/'
else:
    module_path = '/Users/gwonjeong-eul/Desktop/ecp-scl-macbook/ECP_SCL/Experiment(Supervised Learning)/evaluation/'

if module_path not in sys.path:
    sys.path.append(module_path)

import get_hospital_eicu

import split
from imp import reload
reload(split)

import Multiclass_evaluation
reload(Multiclass_evaluation)

<module 'Multiclass_evaluation' from '/Users/gwonjeong-eul/Desktop/ecp-scl-macbook/ECP_SCL/Experiment(Supervised Learning)/evaluation/Multiclass_evaluation.py'>

In [2]:
if os_env == 'window':
    mimic_path = '/Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Case Labeling/mimic_analysis(new_version0313).csv.gz'
    eicu_path = '/Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Case Labeling/eicu_analysis(new_version0313).csv.gz'

else:
    mimic_path = '/Users/gwonjeong-eul/Desktop/ecp-scl-macbook/ECP_SCL/Case Labeling/mimic_analysis(new_version0313).csv.gz'
    eicu_path = '/Users/gwonjeong-eul/Desktop/ecp-scl-macbook/ECP_SCL/Case Labeling/eicu_analysis(new_version0313).csv.gz'

In [82]:
mimic = pd.read_csv(mimic_path, compression = 'gzip')
mimic = mimic.drop('Shock_next_12h', axis = 1)

eicu = pd.read_csv(eicu_path, compression = 'gzip')
eicu = eicu.drop('Shock_next_12h', axis = 1)

dataset_mimic = mimic[~(mimic['gender']==2)].reset_index(drop=True)

dataset_mimic.replace([np.inf, -np.inf], np.nan, inplace=True)
dataset_mimic.fillna(0, inplace=True) 
dataset_mimic = dataset_mimic[~(dataset_mimic['Case']=='event')]
dataset_mimic = dataset_mimic[~((dataset_mimic['INDEX']=='CASE3_CASE4_DF')&(dataset_mimic['Annotation']=='no_circ'))]

eventset = mimic[(mimic['Case']=='event')].reset_index(drop=True)
dataset_mimic['Case'] = pd.to_numeric(dataset_mimic['Case'], errors='coerce')

mimic_train, mimic_valid = data_split(dataset_mimic, 9040, 0.9, Threshold=0.05, n_trial=1, mode = 'mimic')

mimic_train = mimic_train[mimic_train['INDEX']=='CASE3_CASE4_DF'].reset_index(drop=True)
mimic_valid = mimic_valid[mimic_valid['INDEX']=='CASE3_CASE4_DF'].reset_index(drop=True)

eicu_t = eicu_year_process.matching_patient(eicu)
dataset_eicu = eicu_t[~(eicu_t['gender']==2)].reset_index(drop=True)
dataset_eicu.replace([np.inf, -np.inf], np.nan, inplace=True)
dataset_eicu.fillna(0, inplace=True) 

eicu_test = dataset_eicu[~(dataset_eicu['Case']=='event')]
event_eicu = dataset_eicu[(dataset_eicu['Case']=='event')]
eicu_test = eicu_test[~((eicu_test['INDEX']=='CASE3_CASE4_DF')&(eicu_test['Annotation']=='no_circ'))]
eicu_test = eicu_test[eicu_test['INDEX']=='CASE3_CASE4_DF'].reset_index(drop=True)
eicu_test['Case'] = pd.to_numeric(eicu_test['Case'], errors='coerce')

Trial:  0
Threshold 조정 + 0.05, 현재 한계값: 0.1
========== 데이터셋 분할 정보 ==========
데이터셋 비율: 학습 = 0.90, 테스트 = 0.10
학습셋 클래스 비율: Case
1.0    174346
2.0     27608
3.0     70171
4.0     71699
Name: count, dtype: int64
테스트셋 클래스 비율: Case
1.0    17664
2.0     3030
3.0     9456
4.0     8278
Name: count, dtype: int64
--------------------------------------
========== 클래스 비율 ==========
학습셋 클래스 비율: 0.51:0.08:0.20:0.21
테스트셋 클래스 비율: 0.46:0.08:0.25:0.22
--------------------------------------
========== 환자 및 체류 정보 ==========
학습셋 환자 수: 5323
테스트셋 환자 수: 601
학습셋 체류 수: 5433
테스트셋 체류 수: 604
--------------------------------------
========== 실험 설정 ==========
분할 시드: 9040
학습 비율: 0.9
임계값: 0.1
--------------------------------------
========== 실행 결과 ==========
총 소요 시간(초): 0.47
시도한 시행 횟수: 0


In [84]:
if os_env == 'window':
    module_path='/Users/DAHS/Desktop/ECP_CONT/ECP_SCL/Experiment(Supervised Learning)/sub task prediction/'
else:
    module_path = '/Users/gwonjeong-eul/Desktop/ecp-scl-macbook/ECP_SCL/Experiment(Supervised Learning)/sub task prediction'

if module_path not in sys.path:
    sys.path.append(module_path)

import incorperate_subtask_label
reload(incorperate_subtask_label)

<module 'incorperate_subtask_label' from '/Users/gwonjeong-eul/Desktop/ecp-scl-macbook/ECP_SCL/Experiment(Supervised Learning)/sub task prediction/incorperate_subtask_label.py'>

In [110]:
# train, val -> Trainset (underfitting 방지)

mort_mimic_train, mort_mimic_valid, mort_eicu_test = incorperate_subtask_label.mortality_prediction_DATA(mimic_train, mimic_valid, eicu_test, os_env='mac')
mimic_mort_trainset = pd.concat([mort_mimic_train, mort_mimic_valid], axis = 0)

los_mimic_train, los_mimic_valid, los_eicu_test = incorperate_subtask_label.LOS_prediction_DATA(mimic_train, mimic_valid, eicu_test, os_env='mac')
mimic_los_trainset = pd.concat([los_mimic_train, los_mimic_valid], axis = 0)

ards8h_mimic_train, ards8h_mimic_valid, ards8h_eicu_test = incorperate_subtask_label.ARDS8_prediction_DATA(mimic_train, mimic_valid, eicu_test, os_env='mac')
mimic_ards8h_trainset = pd.concat([ards8h_mimic_train, ards8h_mimic_valid], axis = 0)

sic8h_mimic_train, sic8h_mimic_valid, sic8h_eicu_test = incorperate_subtask_label.SIC8_prediction_DATA(mimic_train, mimic_valid, eicu_test, os_env='mac')
mimic_sic8h__trainset = pd.concat([sic8h_mimic_train, sic8h_mimic_valid], axis = 0)

In [111]:
ARDS8h_event_train = ards8h_mimic_train[ards8h_mimic_train['Annotation_ARDS']=='ARDS']
ARDS8h_event_train['ARDS_next_8h']='event'
ARDS8h_event_valid = ards8h_mimic_valid[ards8h_mimic_valid['Annotation_ARDS']=='ARDS']
ARDS8h_event_valid['ARDS_next_8h']='event'
ARDS8h_event_test = ards8h_eicu_test[ards8h_eicu_test['Annotation_ARDS']=='ARDS']
ARDS8h_event_test['ARDS_next_8h']='event'

SIC8h_event_train = sic8h_mimic_train[sic8h_mimic_train['Annotation_SIC']=='SIC']
SIC8h_event_train['SIC_next_8h']='event'
SIC8h_event_valid = sic8h_mimic_valid[sic8h_mimic_valid['Annotation_SIC']=='SIC']
SIC8h_event_valid['SIC_next_8h']='event'
SIC8h_event_test = sic8h_eicu_test[sic8h_eicu_test['Annotation_SIC']=='SIC']
SIC8h_event_test['SIC_next_8h']='event'

In [112]:
X_train_mort, y_train_mort, _ = split.split_X_Y_MORT(mimic_mort_trainset[~(mimic_mort_trainset['death']=='event')], mode = 'mimic')
X_train_los, y_train_los, _ = split.split_X_Y_LOS(mimic_los_trainset, mode = 'mimic')
X_train_ards8h, y_train_ards8h, _ = split.split_X_Y_ARDS8h(mimic_ards8h_trainset, mode = 'mimic')
X_train_sic8h, y_train_sic8h, _ = split.split_X_Y_SIC8h(mimic_sic8h__trainset, mode = 'mimic')

In [113]:
reload(Multiclass_evaluation) 

<module 'Multiclass_evaluation' from '/Users/gwonjeong-eul/Desktop/ecp-scl-macbook/ECP_SCL/Experiment(Supervised Learning)/evaluation/Multiclass_evaluation.py'>

In [114]:
#mortality

lgbm_wrapper = LGBMClassifier(random_state = 42, verbose=-1)
lgbm_wrapper.fit(X_train_mort, y_train_mort)

X_test_mort, y_test_mort, test_output_mort = split.split_X_Y_MORT(mort_eicu_test[~(mort_eicu_test['death']=='event')], mode = 'eicu')

X_test_mort = X_test_mort.drop(['hospitalid', 'hospitaldischargeyear'], axis = 1)

test_preds_mort = lgbm_wrapper.predict(X_test_mort)
test_output_mort['prediction_label'] = test_preds_mort
test_output_mort['prediction_prob'] = lgbm_wrapper.predict_proba(X_test_mort)[:, 1]

test_eval = Multiclass_evaluation.ST_binary_evaluation(_, test_output_mort, 'xgb', mode = 'eicu', event_task = False)
display(test_eval)

,Model,Accuracy,AUPRC,F1 Score,Recall,Precision
0,xgb,0.465397,0.31228,0.415239,0.564418,0.328433


In [116]:
lgbm_wrapper = LGBMClassifier(random_state = 42, verbose=-1)
lgbm_wrapper.fit(X_train_los, y_train_los)

X_test_los, y_test_los, test_output_los = split.split_X_Y_LOS(los_eicu_test, mode = 'eicu')

X_test_los = X_test_los.drop(['hospitalid', 'hospitaldischargeyear'], axis = 1)

test_preds_los = lgbm_wrapper.predict(X_test_los)
test_output_los['prediction_label'] = test_preds_los
test_output_los['prediction_prob'] = lgbm_wrapper.predict_proba(X_test_los)[:, 1]

test_eval_los = Multiclass_evaluation.ST_multi_evaluation(_, test_output_los, 'xgb')
display(test_eval_los)

,Model,Accuracy,Macro AUPRC,F1 Score,Macro averaged precision,Macro averaged Recall
0,xgb,0.6804,0.666667,0.5009,0.572,0.5187


In [121]:
lgbm_wrapper = LGBMClassifier(random_state = 42, verbose=-1)
lgbm_wrapper.fit(X_train_ards8h, y_train_ards8h)

X_test_ards8h, y_test_ards8h, test_output_ards8h = split.split_X_Y_ARDS8h(ards8h_eicu_test, mode = 'eicu')

X_test_ards8h = X_test_ards8h.drop(['hospitalid', 'hospitaldischargeyear'], axis = 1)

test_preds_ards8h = lgbm_wrapper.predict(X_test_ards8h)
test_output_ards8h['prediction_label'] = test_preds_ards8h
test_output_ards8h['prediction_prob'] = lgbm_wrapper.predict_proba(X_test_ards8h)[:, 1]

test_eval_ards8h = Multiclass_evaluation.ST_binary_evaluation(ARDS8h_event_test, test_output_ards8h, 'lgbm', mode='eicu', event_task='ARDS 8h')
display(test_eval_ards8h)

,Model,AUPRC,ARDS score,Recall,Precision,Grecall
0,lgbm,0.161247,0.172566,0.1769,0.245247,0.152482


In [119]:
reload(Multiclass_evaluation) 

eicu_2014 = sic8h_eicu_test[sic8h_eicu_test['hospitaldischargeyear']==2014].drop(['hospitalid', 'hospitaldischargeyear'], axis = 1)
eicu_2015 = sic8h_eicu_test[sic8h_eicu_test['hospitaldischargeyear']==2015].drop(['hospitalid', 'hospitaldischargeyear'], axis = 1)


X_test_sic8h_2014, y_test_sic8h_2014, _ = split.split_X_Y_SIC8h(eicu_2014, mode = 'eicu')
X_test_sic8h_2015, y_test_sic8h_2015, test_output_sic8h = split.split_X_Y_SIC8h(eicu_2015, mode = 'eicu')


X = pd.concat([X_train_sic8h, X_test_sic8h_2014], axis = 0)
Y = np.concatenate((y_train_sic8h, y_test_sic8h_2014))


sic_models, result_sic = Multiclass_evaluation.create_subtask(X, Y, X_test_sic8h_2015, test_output_sic8h, SIC8h_event_test, mode='eicu', type='binary', event_task='SIC 8h')

,Model,AUPRC,SIC score,Recall,Precision,Grecall
0,lgbm,0.36675,0.279232,0.233333,0.450068,0.238925
